In [ ]:
!pip3 install tensorflow_text>=2.0.0rc0


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense


In [ ]:
data=pd.read_csv('/content/drive/MyDrive/mydata.csv')

In [ ]:
data=data.dropna()
data=data.drop('article_content',axis=1)

In [ ]:
max_words = 10000  # Nombre maximum de mots à prendre en compte
max_len = 100  # Longueur maximale d'une séquence

# Tokenisation des articles
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data['cleandata'])
sequences = tokenizer.texts_to_sequences(data['cleandata'])

In [ ]:
labels = pd.get_dummies(data['category']).values

In [ ]:
# Padding des séquences
padded_sequences = pad_sequences(sequences, maxlen=max_len)

# Conversion des catégories en labels numériques
labels = pd.get_dummies(data['category']).values
train_ratio = 0.8  # Pourcentage d'articles à utiliser pour l'entraînement
train_size = int(len(padded_sequences) * train_ratio)
train_data = padded_sequences[:train_size]
train_labels = labels[:train_size]
test_data = padded_sequences[train_size:]
test_labels = labels[train_size:]

In [ ]:
embedding_dim = 100  # Dimension de l'espace d'incorporation des mots

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=max_len))
model.add(LSTM(128))
model.add(Dense(labels.shape[1], activation='softmax'))


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Entraînement du modèle
epochs = 10  # Nombre d'époques d'entraînement
batch_size = 32  # Taille du lot
model.fit(train_data, train_labels, epochs=epochs, batch_size=batch_size, validation_split=0.2)

# Évaluation du modèle sur les données de test
loss, accuracy = model.evaluate(test_data, test_labels, batch_size=batch_size)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

Epoch 1/10
181/181 [==============================] - 45s 231ms/step - loss: 0.5499 - accuracy: 0.7342 - val_loss: 0.2757 - val_accuracy: 0.9218
Epoch 2/10
181/181 [==============================] - 43s 239ms/step - loss: 0.1578 - accuracy: 0.9515 - val_loss: 0.2360 - val_accuracy: 0.9266
Epoch 3/10
181/181 [==============================] - 41s 229ms/step - loss: 0.0914 - accuracy: 0.9759 - val_loss: 0.2304 - val_accuracy: 0.9287
Epoch 4/10
181/181 [==============================] - 42s 230ms/step - loss: 0.0598 - accuracy: 0.9865 - val_loss: 0.2395 - val_accuracy: 0.9308
Epoch 5/10
181/181 [==============================] - 42s 234ms/step - loss: 0.0321 - accuracy: 0.9910 - val_loss: 0.2575 - val_accuracy: 0.9329
Epoch 6/10
181/181 [==============================] - 43s 239ms/step - loss: 0.0288 - accuracy: 0.9933 - val_loss: 0.2593 - val_accuracy: 0.9301
Epoch 7/10
181/181 [==============================] - 42s 230ms/step - loss: 0.0148 - accuracy: 0.9964 - val_loss: 0.3691 - val_ac

In [ ]:
# Supposons que vous avez une nouvelle phrase en arabe à classifier
new_text="القاهرة- لم يصدق عمر الشاهد ما يثار على وسائل التواصل بشأن الارتفاع الجنوني في أسعار اللحوم، وذهب بنفس رر: هل يشتري الأضحية هذا العام أم يصرف عنها النظر"
# Prétraitement de la nouvelle phrase
new_sequence = tokenizer.texts_to_sequences([new_text])
new_padded_sequence = pad_sequences(new_sequence, maxlen=max_len)

# Faire la prédiction
predictions = model.predict(new_padded_sequence)

# Convertir les probabilités en catégories
predicted_category = np.argmax(predictions, axis=1)

# Récupérer le nom de la catégorie prédite
category_labels = data['category'].unique()
predicted_category_label = category_labels[predicted_category[0]]

print("Phrase :", new_text)
print("Catégorie prédite :", predicted_category_label)


1/1 [==============================] - 1s 664ms/step
Phrase : القاهرة- لم يصدق عمر الشاهد ما يثار على وسائل التواصل بشأن الارتفاع الجنوني في أسعار اللحوم، وذهب بنفس رر: هل يشتري الأضحية هذا العام أم يصرف عنها النظر
Catégorie prédite : اقتصاد
